In [5]:
import pandas as pd

# Read data from CSV files into DataFrames
brands = pd.read_csv("D:/Python Projects/Analyzing Online Sports Revenue/brands.csv") 
finance = pd.read_csv("D:/Python Projects/Analyzing Online Sports Revenue/finance.csv")
info = pd.read_csv("D:/Python Projects/Analyzing Online Sports Revenue/info.csv")
reviews = pd.read_csv("D:/Python Projects/Analyzing Online Sports Revenue/reviews.csv")

# Merge the DataFrames based on the "product_id" using outer joins
online_sports_revenue = brands.merge(finance,
                                     on='product_id',
                                     how='outer')
online_sports_revenue = online_sports_revenue.merge(info,
                                                    on='product_id',
                                                    how='outer')
online_sports_revenue = online_sports_revenue.merge(reviews,
                                                    on='product_id',
                                                    how='outer')

# Remove rows with missing values
online_sports_revenue.dropna(inplace=True)

# Define quartile labels
quartile_labels = ['Budget', 'Average', 'Expensive', 'Elite']

# Create a new column "price_label" based on "listing_price" quartiles
online_sports_revenue['price_label'] = pd.qcut(
    online_sports_revenue['listing_price'],
    q=4,
    labels=quartile_labels)

# Group data by "brand" and "price_label," calculating counts and mean revenue
adidas_vs_nike = online_sports_revenue.groupby(
    ['brand', 'price_label'],
    as_index=False).agg(num_products=('price_label',
                                      'count'),
                        mean_revenue=('revenue',
                                      'mean')).round(2).reset_index(drop=True)

# Calculate the maximum description length
max_description_length = max(online_sports_revenue['description'].str.len())

# Calculate description lengths
online_sports_revenue['description_length'] = online_sports_revenue['description'].str.len()

# Define bins and labels for "description_length"
lengthes = [0, 100, 200, 300, 400, 500, 600, 700]
labels = ['100', '200', '300', '400', '500', '600', '700']

# Apply binning to "description_length" column
online_sports_revenue['description_length'] = pd.cut(
    online_sports_revenue['description_length'],
    bins=lengthes,
    labels=labels)

# Group by "description_length" and calculate mean rating and number of reviews
description_lengths = online_sports_revenue.groupby(
    'description_length',
    as_index=False).agg(mean_rating=('rating',
                                     'mean'),
                        num_reviews=('reviews',
                                     'count')).round(2)

# Define a regex pattern for filtering footwear products in "description"
footwear_list = 'shoe*|trainer*|foot*'

# Create DataFrames for footwear and clothing products
shoes = online_sports_revenue[online_sports_revenue['description'].str.contains(footwear_list)]
clothing = online_sports_revenue[~online_sports_revenue.isin(shoes['product_id'])]

# Remove rows with missing values in the clothing DataFrame
clothing.dropna(inplace=True)

# Create a summary DataFrame with product type statistics
product_types = pd.DataFrame(
    {'num_clothing_products': len(clothing),
     'median_clothing_revenue': clothing['revenue'].median(),
     'num_footwear_products': len(shoes),
     'median_footwear_revenue': shoes['revenue'].median()},
    index=[0])

print(product_types)


   num_clothing_products  median_clothing_revenue  num_footwear_products  \
0                    478                   625.07                   2639   

   median_footwear_revenue  
0                   3073.3  
